In [2]:
import numpy as np
from Poisson import *
import math as mt
from dataset import *
import mpmath
from Time_series import *
from team import *
from Tests import * 

Setting Precision of Mpmath:

In [10]:
mpmath.mp.dps=10

print(mpmath.mp)

Mpmath settings:
  mp.prec = 53                [default: 53]
  mp.dps = 15                 [default: 15]
  mp.trap_complex = False     [default: False]


Checking Gradient of log

In [11]:
gradcheck()

Success


Loading the dataset:

In [12]:
Dataset=FootballDataset('E0_2022.csv')
data=Dataset.data
team_names=Dataset.team_names()

Static/Semi-Dynamic Estimation:


In [13]:
from scipy.optimize import minimize
import numpy as np
import math
np.seterr(over='raise')



def static_semidynamic_compute(fphi,data,t,Weighting=False):
    """
    Static and Semi-Dynamic Computation of Parameters
    """

    f = fphi[:40]
    phi = fphi[40:]
    s = 0
    k=0
    nu=0.1
    Log_loss=0.
    for i in range(t):
        home = data[i]['HomeTeam']
        away = data[i]['AwayTeam']
        jh = int([k for k in range(1, 21) if team_names[k] == home][0]) - 1
        ja = int([k for k in range(1, 21) if team_names[k] == away][0]) - 1
        x = int(data[i]['FTHG'])
        y = int(data[i]['FTAG'])
        alpha_i, alpha_j, beta_i, beta_j = f[jh], f[ja], f[jh + 20], f[ja + 20]
        lam1 = mpmath.exp(phi[0] + alpha_i - beta_j)
        lam2 = mpmath.exp(alpha_j - beta_i)
        lam3 = phi[1]
        Log_loss+= -mpmath.log(density(x, y, lam1, lam2, lam3))

        if Weighting:
            if i % 10 == 0:
                k+=1
                s += (mpmath.exp(-nu*k))*Log_loss
                Log_loss=0

    return s

def constraint_func(fphi):
    f = fphi[:40]
    phi = fphi[40:]
    return np.sum(f[:20])  # Sum of first 20 elements of f should be zero, phi[1] should be positive

# define the initial guess for the parameters
fphi_init = np.ones(42)
fphi_init[:10] = 1/2 * np.ones(10)
fphi_init[10:20] = -1/2 * np.ones(10)

# define the constraints
constraints = [{'type': 'eq', 'fun': constraint_func}]

# define the bounds
bounds = [(None,None)] * 40 + [(0, None), (0.001, None)]

# use the minimize function to minimize the static_compute function
result = minimize(static_semidynamic_compute, fphi_init, args=(data), constraints=constraints, bounds=bounds)

In [14]:
bestat=np.argmax(result.x[:20])
print(team_names[bestat+1])
print(np.argmax(result.x[20:40]))
print(result.x)
f0=(result.x[:40]).copy()


Liverpool
16
[ 0.08669425  0.14118779  0.43085673  0.06757545 -0.07175179 -0.47744926
 -0.80472452  0.17331634 -0.58784355 -1.19326657 -0.35590384  0.88754268
  0.16193841 -0.06113368 -0.31539851 -0.19011739  0.69742143  0.29051178
  0.40523473  0.71530953  0.12135144 -0.0734551   0.13070563  0.29539287
  0.35556113  0.30812893  0.31552493  0.23251046  0.68528673 -0.30815123
  0.52745885  0.95926371  0.11602023 -0.27517513 -0.24168406 -0.31324698
  1.54543524 -0.14169365  0.27997708  1.49103276  0.36270366  0.21896959]


Dynamic Model

In [37]:
def compute_scores(data,team_names,t,f0,phi):
    """
    Dynamic Estimation of parameters
    
    """
  
    Teams=[Team_score(team_names,i,f0[i-1],f0[19+i]) for i in range(1,21)]
    ###f0 shape 2N
    B=Mat_A(phi[2],phi[3])
    omega=(1-np.diag(B)) * f0
    sig=phi[4]
    lam3=phi[5]
    s=0
    f=f0.copy()
    for i in range(21,t):
        Home= data[i]['HomeTeam']     #### Home Team name
        Away= data[i]['AwayTeam'] 
        jh=int([k for k in range(1,21) if team_names[k]== Home ][0]) -1   
        
        ja=int([k for k in range(1,21) if team_names[k]== Away ][0]) -1 
        
        x=int(data[i]['FTHG'] )            
        y=int(data[i]['FTAG'] )
        
        M_ij=Mat_M(jh,ja)
        
        omega_ij= M_ij @ omega
        
        f_ij=update_score(jh,ja,x,y,f,phi,omega_ij)
        
        Teams[jh].update(f_ij[0],f_ij[2])
        Teams[ja].update(f_ij[1],f_ij[3])

        lam1= mpmath.exp(sig+f_ij[0]-f_ij[2])
        lam2= mpmath.exp(f_ij[1]-f_ij[3])
        
        s += -mpmath.log(density(x, y, lam1, lam2, lam3))
        
        f[jh],f[jh+20],f[ja],f[ja+20] = f_ij
   
    return f,s
        

In [52]:
f0=result.x[:40].copy()
print(f0)

[  5.89390576   6.86175966   4.46854598   6.63572083 -20.69680565
 -21.66511947   4.92606334  50.00029352 -14.86969979 -21.93979721
  20.47530533   5.50592999   6.76762494   8.52301523   5.89615426
   6.71481146   6.17654275 -13.89884518 -33.05018036 -12.72522539
  50.9696342   20.3453708  -19.72845127   6.57965435  30.46640004
   8.10531841  96.50573175   5.20125704   6.73948486   7.13697423
   6.16911576  25.96718389  28.75314372 -21.66410323   6.59814601
   6.29927954   6.0027663    5.93852899   3.82745617 -20.69678617]


In [88]:
team_names

{1: 'Man City',
 2: 'Chelsea',
 3: 'Burnley',
 4: 'Norwich',
 5: 'Aston Villa',
 6: 'Tottenham',
 7: 'Everton',
 8: 'Crystal Palace',
 9: 'Southampton',
 10: 'Newcastle',
 11: 'West Ham',
 12: 'Brentford',
 13: 'Liverpool',
 14: 'Arsenal',
 15: 'Brighton',
 16: 'Watford',
 17: 'Wolves',
 18: 'Man United',
 19: 'Leicester',
 20: 'Leeds'}

In [95]:
f=np.zeros(40)
f[:20]=0.44*np.ones(20)
def objective_function(phi):
    _, s = compute_scores(Dataset, data, team_names, 380,0.44* np.ones(40), phi)
    return -s  # Negate s to convert maximization to minimization

# Define the bounds for phi elements (positive bounds)
bounds = [(0.01, 0.1)] * 2+[(0.5, 0.99)] * 2+[(0.1, 0.44)] + [(0.01,0.1)]# Assuming all elements of phi have positive bounds

# Perform optimization
initial_phi = np.array([0.01,0.01,0.9,0.9,0.36,0.1])  # Initial guess for phi
result = minimize(objective_function, initial_phi, bounds=bounds)

# Check if optimization was successful
if result.success:
    optimal_phi = result.x
    _, optimal_s = compute_scores(Dataset, data, team_names, 380,f, optimal_phi)
    print("Optimization successful!")
    print("Optimal phi:", optimal_phi)
    print("Optimal second variable (s):", optimal_s)
else:
    print("Optimization failed:", result.message)

Optimization successful!
Optimal phi: [0.01 0.1  0.5  0.99 0.1  0.01]
Optimal second variable (s): 1378.07560556706


In [96]:
L,s=compute_scores(Dataset, data, team_names, 380, f, optimal_phi)
print(L)

[0.77433875 0.5324688  0.63244738 0.6647909  1.03205459 0.69073612
 1.14344742 0.57393251 1.2510292  1.1722182  0.9805599  0.8528959
 0.61833709 0.70492561 0.65764216 0.67346783 1.31070785 1.25351065
 0.82492775 0.9251269  0.49253741 0.53436381 0.67023813 0.56266125
 0.7566932  0.83567697 0.51646588 0.9547779  0.67204331 1.01280085
 0.60484307 0.54327959 0.53652394 0.53705378 0.49654174 0.71531426
 0.71511099 0.75283645 0.55114621 0.9336616 ]
